<a href="https://colab.research.google.com/github/hjtb/Data-Validation/blob/main/Product_Validation_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRODUCT VALIDATION SCRIPT
# Authors - David Leon (@Dleon) and William Holton (@Wholton)
## Intro
### Validations:

1.  Product URL needs to start with https:// or http://
2.  Company URL starts with https://www.linkedin.com/company/
3.  Company URL needs to be in the format of https://www.linkedin.com/company/<vanity_name> instead of [..]<company_id>
4.  Showcase page URL needs to start with https://www.linkedin.com/showcase/<vanity_name>
5.  Product Category ID corresponds to category/categories assigned to product
6.  Ensure Product URL is NOT a PDF (i.e. does not end with ‘.pdf’)
7.  Add validation to ensure product categories assigned to products are NOT Group Representatives **(multi-coded)
8.  Identify required fields and add a check to ensure all of those fields are filled.
9. Validation that isActive = T, and isDeprecated = F for New Products, and the opposite deprecations on Product Changes. Also, check that both values are not T/T or F/F.
10. Ensure all characters are unicode. (i.e. No special characters like Äô)
11. Ensure Product Skill ID is valid and not 0
12. Make sure there are no line breaks in product names or descriptions
13.  Check that 'Showcase URL' and 'Showcase ID' are not placed in each others field
14. Check that product name and product ID match for product changes
13. Check that showcase IDs are within the valid range **[WIP]**
15. Dupe checks **[WIP]**
16. Add comments for different error types **[Added to 1st 14]**

### Work on gathering Summary Statistics of the errors caught by the script [WIP]:


*   Number of errors caught by each validation
*   Storing this data somewhere to have a history of errors and statistics on them.







## Install and import relevant packages (restart runtime after installation)

In [1]:
# If you encounter the AttributeError: 'NotebookFormatter' object has no attribute 'get_result' run this and then reinstall the packages
# !pip uninstall numpy
# !pip uninstall pandas

In [2]:
# Install pygsheets to be able to connect to the spreadsheet:
!pip install pygsheets &> /dev/null 
!pip install validators
!pip install pandas==1.3  # We will need this version to use explode function on multiple columns
!pip install xlsxwriter

     |████████████████████████████████| 10.8 MB 4.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
     |████████████████████████████████| 149 kB 4.3 MB/s 


In [4]:
import pygsheets
import numpy as np
import pandas as pd
from pygsheets.datarange import DataRange
import pprint
import validators
import string
import xlsxwriter 

## Get Credentials and set the Script Mode

In [6]:
# Get connection through Service Account credentials (Google APIs):

try:
  # (EL 1) David Credentials
  google_credentials = pygsheets.authorize(service_account_file=
                         './product-data-validation-5a7794651578.json')
except:
  # (EL 2) Will Credentials
  google_credentials = pygsheets.authorize(client_secret='./client_secret_will.json')

Please go to this URL and finish the authentication flow: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=956387985412-hbneo3nm10jmucu9epq2ugineplnc907.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=bytZQvhnN5AQlVv68lI4pAvwxIAeh6&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWgredFuOvlQc0JHsPirPFa1AaHErPAMiW0cjuIf49xAebxu3IU7TXU


In [7]:

# Ask for input about the 1st mode the script is going to run in:
script_mode_1 = input("Are you validating NEW PRODUCTS or PRODUCT CHANGES? Answer with: 'new products'/'product changes'")

# Keep asking for input until a valid input is provided
if (script_mode_1 == 'new products' or script_mode_1 == 'product changes') == False:
    while True:
        script_mode_1 = input("A typo was probably inputted. Please answer with:'new products'/'product changes'.")
        if (script_mode_1 == 'new products' or script_mode_1 == 'product changes') == True:
            break

# Ask for input about the second mode the script is going to run in:
script_mode_2 = input("Are you validating the CATALOG + PIPELINE/Product changes sheet or the PUBLICATION sheets? \nAnswer with: 'pipeline'/'publication'")

# Keep asking for input until a valid input is provided
if (script_mode_2 == 'pipeline' or script_mode_2 == 'publication') == False:
    while True:
        script_mode_2 = input("A typo was probably inputted. \nPlease answer with:'pipeline'/'publication'.")
        if (script_mode_2 == 'pipeline' or script_mode_2 == 'publication') == True:
            break

# SOL N1: -------------------------------------------------------------------------------------------------------------------


# Handle product changes if the input points to that script mode:
if script_mode_1 == 'product changes' and script_mode_2 == 'pipeline':
    spreadsheet_to_be_validated = google_credentials.open('Product Changes')
    specific_to_be_validated_tab = spreadsheet_to_be_validated.worksheet_by_title('Changes')

elif script_mode_1 == 'product changes' and script_mode_2 == 'publication':
    spreadsheet_to_be_validated = google_credentials.open('[Template] Changes to Products')
    specific_to_be_validated_tab = spreadsheet_to_be_validated.worksheet_by_title('Future Product Changes')

# Handle new products option:
elif script_mode_1 == 'new products':

    # If the chosen mode is for the publication sheet (script_mode_2 = 'publication'):
    if script_mode_2 == 'publication':
        spreadsheet_to_be_validated = google_credentials.open('[Template] Changes to Products')
        specific_to_be_validated_tab = spreadsheet_to_be_validated.worksheet_by_title('Future New Products')
    
    # If the chosen mode is not for the publication sheet (script_mode_2 = 'pipeline'):
    elif script_mode_2 == 'pipeline':
        # Ask for the period tab we'll be using for the Catalog + Pipeline sheet:  
        period_tab = input("Introduce the Catalog + Pipeline period tab you'd like to be testing. e.g. 'dev' or 'FY22Q2 Review Period 2 (08/11-07/12)':")

        while True:

            # Go into development mode:
            if period_tab == 'dev':
              spreadsheet_to_be_validated = google_credentials.open('Catalog + Product Pipeline')
              specific_to_be_validated_tab = spreadsheet_to_be_validated.worksheet_by_title('Validation Script Dev')
              print(f"Period tab chosen is: {period_tab}")
              break

            # Otherwise look for a specific tab:
            else:
              # try clause to check if the name of the tab provided exists:
              try:
                spreadsheet_to_be_validated = google_credentials.open('Catalog + Product Pipeline')
                specific_to_be_validated_tab = spreadsheet_to_be_validated.worksheet_by_title(period_tab)
                print(f"Period tab chosen is: '{period_tab}'")
                break
              except:
                period_tab = input("A typo was probably inputted. \nPlease try to introduce the period tab name again:")
                pass


script_mode_1, script_mode_2

Are you validating NEW PRODUCTS or PRODUCT CHANGES? Answer with: 'new products'/'product changes'new products
Are you validating the CATALOG + PIPELINE/Product changes sheet or the PUBLICATION sheets? 
Answer with: 'pipeline'/'publication'pipeline
Introduce the Catalog + Pipeline period tab you'd like to be testing. e.g. 'dev' or 'FY22Q2 Review Period 2 (08/11-07/12)':dev
Period tab chosen is: dev


('new products', 'pipeline')

## Create main Data Objects

In [8]:
# Convert Pipeline sheet into pandas dataframe:
dataframe_to_be_validated = specific_to_be_validated_tab.get_as_df()
dataframe_to_be_validated.index += 2  # Shift indexes to match the original doc


# Get non-empty rows
product_names = dataframe_to_be_validated.loc[:, 'Product name']
dataframe_to_be_validated = dataframe_to_be_validated.loc[product_names.str.len().gt(0)]

dataframe_to_be_validated.head(2)

,Collected by,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Description,Active,Deprecated,...,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error Type,Reviewer Comments,productType
2,David,08/10/21,1127,Email Security Software,,Paubox Email Suite\n,,À È Ì Ò Ù Ỳ Ǹ Ẁ\t`A\tLetter with grave accent\...,,,...,,,,,Yes,,FALSE,,,
3,David,08/10/21,1238,Infrastructure as a Service (IaaS),,Alestra Redes Administradas,,,,,...,,,Yes,,Yes,,FALSE,,,


In [9]:
# Open Golden Category Status spreadsheet:
golden_spreadsheet = google_credentials.open('Golden Category Status ')
specific_golden_tab = golden_spreadsheet.worksheet_by_title('Golden Product Categories')

In [10]:
# Convert Golden sheet into pandas dataframe:
golden_sheet_dataframe = specific_golden_tab.get_as_df()
golden_sheet_dataframe.head(2)

/usr/local/lib/python3.7/dist-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


,,URN,Category Name,Category Description,Aliases,See Also,UPDATED Proposed: Parent Problem Category,parent_product_category_ids,Included in MVP,active,...,Example Product,Category Created by,Has Metadata,Evidence,Category in G2?,Only in G2,Notes,,new count,
0,,,,,,,,,524,,...,,,,,71%,14%,,,,
1,1,1000,Enterprise Accounting Software,Software used to record and process financial ...,,,Financial Management > Accounting,10129,YES,T,...,,Erin,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,TRUE,FALSE,"Previously called ""Accounting Software"" but re...",,Unique words,Frequency


In [11]:
# Open Catalog spreadsheet:
prod_catalog_spreadsheet = google_credentials.open('STZ Dedupe Check')
catalog_tab = prod_catalog_spreadsheet.worksheet_by_title('Product Catalog & Admin Products')

In [12]:
# Convert Catalog sheet into pandas dataframe:
product_catalog_dataframe = catalog_tab.get_as_df()
product_catalog_dataframe.head

<bound method NDFrame.head of        productId                    productName  \
0           1000  QuickBooks Desktop Enterprise   
1           1001                       Switchit   
2           1002                       MBizCard   
3           1003                   Sage Intacct   
4           1004                           Xero   
...          ...                            ...   
86664    1735471                         CERQEL   
86665    1735472                         TREMVO   
86666    1735477         Création site internet   
86667    1735518   Goldshell KD5 Miner for sale   
86668    1810547                       BombBomb   

                                              productUrl  
0      https://quickbooks.intuit.com/desktop/enterprise/  
1                               https://switchitapp.com/  
2                                  https://mbizcard.com/  
3                                        sageintacct.com  
4                                   https://www.xero.com  
...

# Validations


## Highlight the cells with errors in red and (add notes to the errors) (Function) **In progress[Will]**


In [14]:
def highlight_errors(error_indexes_array,column_index, error):
  for row in error_indexes_array:
    # Select the cell we wish to change the color of using the row index and the column value O for product Urls
    error_cell = specific_to_be_validated_tab.cell(f'{column_index}{row}')
    # Change the error cells to red
    error_cell.color = (1, 0.1, 0.1, 0.5)
    #Highlight the ELs name in green so they can locate the rows with errors (Pipeline sheet only)
    if script_mode_1 == 'product changes' and script_mode_2 == 'pipeline':
      el_cell = specific_to_be_validated_tab.cell(f'B{row}')
    elif script_mode_2 == 'publication':
      el_cell = specific_to_be_validated_tab.cell(f'Y{row}')
    elif script_mode_1 == 'new products' and script_mode_2 == 'pipeline':
      el_cell = specific_to_be_validated_tab.cell(f'A{row}')

    el_cell.color = (0.1, 0.98, 0.4, 0.0001)
    # Add the appropriate error note to the cell
    if error == "prod_url_error":
      error_cell.note = "Check that there is no whitespace either side of the URL and that the URL starts with http/https"
    if error == "company_url_vanity_name":
      error_cell.note = "Company URL needs to be in the format of https://www.linkedin.com/company/'vanity_name' ensure that the URL doesn't contain the company ID"
    if error == "company_url":
      error_cell.note = "Company URL must start with 'https://www.linkedin.com/company/'(and the company name) make sure to check that there is no whitespace either side of the URL"
    if error == "showcase_url":
      error_cell.note = "Showcase URL must start with 'https://www.linkedin.com/showcase/' make sure to check that there is no whitespace either side of the URL"
    if error == "single_coded_category_error":
      error_cell.note = "Ensure the category name matches the category ID and that there is no whitespace either side of the category name. Also, check there are no spelling mistakes. Avoid typing the category names, always copy and paste directly from the golden sheet"
    if error == "double_coded_category_error":
      error_cell.note = "Ensure the category names match the IDs and that there is no whitespace in between names and commas. Multicoded must be in the format 'category_name_1,category_name_2,category_name_3' Also, check there are no spelling mistakes."
    if error == "pdf_error":
      error_cell.note = "Product URLs can't end in .pdf"
    if error == "group_rep":
      error_cell.note = "Products can't be assigned to group representatives. ie. Categories highlighted in blue on the golden sheet"
    if error == "active_deprecated":
      error_cell.note = "Ensure the correct values for Active and Deprecated are entered. New products should have Active = 'T' and Deprecated = 'F'. If you are deprecating the product the opposite applies. F or T for both results in an error"
    if error == "skill_id":
      error_cell.note = "Skill ID must be between between 1 and 65000"
    if error == "required":
      error_cell.note = "This field is required"
    if error == "linebreak":
      error_cell.note = "There is a linebreak in this cell, if this is intentional ignore it. Otherwise remove it."
    if error == "showcase_id":
      error_cell.note = "Ensure this is a valid showcase ID"
    if error == "product_changes_name_id":
      error_cell.note = "If this is a product name change, ignore it. If not, please check the product name is correct"

## 1.  Product URL needs to start with https:// or http://



In [21]:
# First get the column of product URLs
product_urls = dataframe_to_be_validated['Product URLS']
product_urls_dataframe = product_urls.to_frame()

#### Use Validators method to validate URLs

In [22]:
# Use validators package to validate urls and assign true and false values in new column called isURLValid
def isUrlValid(url):
    return True if validators.url(url) else False
product_urls_dataframe['isURLValid'] = product_urls_dataframe['Product URLS'].apply(isUrlValid)

In [23]:
# Get rows where url is not deemed valid
product_url_errors_validator_method = product_urls_dataframe.loc[product_urls_dataframe['isURLValid'] == False]
product_url_errors_validator_method

,Product URLS,isURLValid
332,www.pwc.com/us/en/products/risk-proof.html,False
339,,False
340,,False
414,arda.ai/the-product/,False


#### Use pandas method to validate URLs

In [24]:
# Get the rows that don't pass the 'http'/'https' validation criteria:
prod_url_errors_string_method = dataframe_to_be_validated.loc[product_urls.str.startswith('http','https') == False, :]
prod_url_errors_string_method

,Collected by,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Description,Active,Deprecated,...,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error Type,Reviewer Comments,productType
332,Nicole,2021-08-31,1210,Anti-Money Laundering (AML) Software,,PWC Risk Proof,,,T,F,...,,,Yes,,,,FALSE,,,
339,Eimear,2021-09-01,1090,Darabase as a Service (DBaaS),,Oracle Database,,,T,F,...,,,Yes,Yes,,,FALSE,,Dupe of Product 12189,
340,Eimear,2021-09-01,1245,Master Data Management (MDM) Software,,Oracle Enterprise Data Management Cloud,,,T,F,...,,,,Yes,,Will,TRUE,,Dupe of 16821 Oracle Enterprise Data Management,
414,Nicole,2021-09-08,1805,AI-Powered Coaching Software,,ARDA AI Coaching Platform,,,F,F,...,,,Yes,,,,FALSE,,,


#### Group errors from both methods in a set of unique index values and highlight the corresponding errors

In [25]:
all_prod_url_errors_index = product_url_errors_validator_method.index.append(prod_url_errors_string_method.index)
all_prod_url_errors_index = set(all_prod_url_errors_index)
all_prod_url_errors_index

{332, 339, 340, 414}

In [26]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Product URLS") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "prod_url_error"
highlight_errors(all_prod_url_errors_index, column_letter, error)

## 2. Company URL starts with "https://www.linkedin.com/company/" (and the company name)


In [27]:
# get non-empty showcase urls by checking if their string length is greater than 0
#company_urls_not_empty = dataframe_to_be_validated.loc[company_urls.str.len().gt(0)]
company_urls_not_empty = dataframe_to_be_validated[dataframe_to_be_validated['Comp URL'].str.len().gt(0)]
company_urls_not_empty

,Collected by,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Description,Active,Deprecated,...,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error Type,Reviewer Comments,productType
2,David,08/10/21,1127,Email Security Software,,Paubox Email Suite\n,,À È Ì Ò Ù Ỳ Ǹ Ẁ\t`A\tLetter with grave accent\...,,,...,,,,,Yes,,FALSE,,,
3,David,08/10/21,1238,Infrastructure as a Service (IaaS),,Alestra Redes Administradas,,,,,...,,,Yes,,Yes,,FALSE,,,
4,David,08/10/21,1037,Marketing Automation Software,,emBlue,,,T,T,...,,,,,Yes,,FALSE,,,
5,David,08/10/21,1780,Virtual Fitting Software,,3dmeasureup,,,F,F,...,,,Yes,,Yes,,FALSE,,,
6,David,08/10/21,1780,Virtual Fitting Software,,3dbodyscan\n,,,TRUE,TRUE,...,,,,,Yes,Will,TRUE,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,Harry,2021-09-09,164216431654,Supply Side Platforms (SSP) AND Demand Side Pl...,,Index Exchange,,,T,F,...,,,Yes,,Yes,,FALSE,,Dupe of Product 17460,
453,Harry,2021-09-09,1001,Campaign Management Software,,Campaign Manager +,,,T,F,...,,,Yes,,Yes,,FALSE,,Dupe of Product 18649,
454,Harry\n,2021-09-09,1244,E-Commerce Analytics Software,,Beeketing,,,T,F,...,,,Yes,,Yes,,FALSE,,,
455,Harry,2021-09-09,1615,Web Hosting\n,,alfahosting Web Hosting,,,T,F,...,,,Yes,,Yes,,FALSE,,,


In [28]:
# Get the company urls column
company_urls = company_urls_not_empty.loc[:, 'Comp URL']
company_urls.head()

2                    https://www.linkedin.com/company/
3        https://www.linkedin.com/company/11695/admin/
4         https://www.linkedin.com/company/embluemail/
5    https://www.linkedin.com/company/prototech-sol...
6    https://www.linkedin.com/company/vitronic-dr--...
Name: Comp URL, dtype: object

In [29]:
# get all company urls that don't start with https://www.linkedin.com/company/
comp_url_errs_string_method = company_urls_not_empty.loc[(company_urls.str.startswith('https://www.linkedin.com/company/') == False) | ((company_urls.str.len() == (len('https://www.linkedin.com/company/') - company_urls.str.count(' '))) == True)]
# count how many urls are incorrect
comp_url_errs_string_method_index = comp_url_errs_string_method.index
comp_url_errs_string_method_index

Int64Index([2, 32, 35, 37, 93, 105, 158, 159, 160, 380, 392, 454], dtype='int64')

#### Use Validators method to validate URLs

In [30]:
# Use validators package to validate urls and assign true and false values in new column called isURLValid
company_urls_dataframe = company_urls.to_frame()
def isUrlValid(url):
    return True if validators.url(url) else False
company_urls_dataframe['isURLValid'] = company_urls_dataframe['Comp URL'].apply(isUrlValid)

In [31]:
# Get rows where url is not deemed valid
comp_url_errs_validators_method = company_urls_dataframe.loc[company_urls_dataframe['isURLValid'] == False]
comp_url_errs_validators_method_index = comp_url_errs_validators_method.index
comp_url_errs_validators_method_index

Int64Index([], dtype='int64')

#### Group errors from both methods in a set of unique values

In [32]:
all_comp_url_errors_index = comp_url_errs_validators_method_index.append(comp_url_errs_string_method_index)
all_comp_url_errors_index_unique = set(all_comp_url_errors_index)
all_comp_url_errors_index

Int64Index([2, 32, 35, 37, 93, 105, 158, 159, 160, 380, 392, 454], dtype='int64')

#### Highlight the cells with errors in red

In [33]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Comp URL") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "company_url"
highlight_errors(all_comp_url_errors_index, column_letter, error)

## 3. Company URL needs to be in the format of `https://www.linkedin.com/company/'vanity_name'` instead of `[..]'company_id'`


In [34]:
# Get the part of the url that comes after the 'company/' address:
vanity_name_bit = company_urls.str.split('https://www.linkedin.com/company/', expand=True)
vanity_name_bit.head()

,0,1
2,,
3,,11695/admin/
4,,embluemail/
5,,prototech-solutions-&-services/
6,,vitronic-dr--ing-stein-bildverarbeitungssystem...


In [35]:
# Keep just the vanity name part and turn to series object:
vanity_name_bit = vanity_name_bit.pop(1).squeeze()  
# In case someone has added the url with the id both from admin view, or for member view:
vanity_name_bit = vanity_name_bit.str.split('/admin', expand=True).pop(0).squeeze()
vanity_name_bit = vanity_name_bit.str.split('/mycompany', expand=True).pop(0).squeeze()
vanity_name_bit.head()

2                                                     
3                                                11695
4                                          embluemail/
5                      prototech-solutions-&-services/
6    vitronic-dr--ing-stein-bildverarbeitungssystem...
Name: 0, dtype: object

In [36]:
# Get rid of forward slash or potential white spaces at the end so that it doesn't 
# interfere with isdigit() in checking if the string is only numbers:
vanity_name_bit = vanity_name_bit.str.rstrip('/ ')
vanity_name_bit.head()

2                                                     
3                                                11695
4                                           embluemail
5                       prototech-solutions-&-services
6    vitronic-dr--ing-stein-bildverarbeitungssystem...
Name: 0, dtype: object

In [37]:
# Check if the url corresponding with the vanity name is only numbers (i.e. company ID instead of vanity name):
vanity_name_errs = company_urls_not_empty.loc[vanity_name_bit.str.isdigit() == True, :]
vanity_name_errs_index = vanity_name_errs.index
vanity_name_errs_index

Int64Index([3, 45, 60, 100], dtype='int64')

#### Highlight the cells with errors in red

In [38]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Comp URL") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "company_url_vanity_name"
highlight_errors(vanity_name_errs_index, column_letter, error)

## 4. Showcase page URL needs to start with https://www.linkedin.com/showcase/<vanity_name>  and is deemed a valid url by the Validators package

In [45]:
# get the showcase urls
showcase_urls = dataframe_to_be_validated.loc[:, 'Showcase URL']
showcase_urls.head()

2    
3    
4    
5    
6    
Name: Showcase URL, dtype: object

In [46]:
# get non-empty showcase urls by checking if their string length is greater than 0
showcase_col_not_empty = dataframe_to_be_validated.loc[showcase_urls.str.len().gt(0)]
non_empty_showcase_urls = showcase_col_not_empty['Showcase URL']
non_empty_showcase_urls

45     https://www.linkedin.com/showcase/quickbooks/
51           https://www.linkedin.com/in/heathadams/
239     https://www.linkedin.com/showcase/contensis/
Name: Showcase URL, dtype: object

In [47]:
# find showcase url errors where they don't begin with 'https://www.linkedin.com/showcase/'
showcase_url_errs = showcase_col_not_empty.loc[non_empty_showcase_urls.str.startswith('https://www.linkedin.com/showcase/') == False]
showcase_url_errs = dataframe_to_be_validated.loc[showcase_url_errs.index]
showcase_url_errs_index = showcase_url_errs.index

#### Use Validators method to validate URLs

In [48]:
# Use validators package to validate urls and assign true and false values in new column called isURLValid
non_empty_showcase_urls_dataframe = non_empty_showcase_urls.to_frame()
def isUrlValid(url):
    return True if validators.url(url) else False
non_empty_showcase_urls_dataframe['isURLValid'] = non_empty_showcase_urls_dataframe['Showcase URL'].apply(isUrlValid)

In [49]:
# Get rows where url is not deemed valid
validator_showcase_url_errs = non_empty_showcase_urls_dataframe.loc[non_empty_showcase_urls_dataframe['isURLValid'] == False]
validator_showcase_url_errs_index = validator_showcase_url_errs.index
showcase_url_errs_index.append(validator_showcase_url_errs_index)

Int64Index([51], dtype='int64')

#### Highlight the cells with errors in red

In [50]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Showcase URL") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "showcase_url_err"
highlight_errors(showcase_url_errs_index, column_letter, error)

## 5. Product Category ID corresponds to category/categories assigned to product

### Single Coded Products:

In [51]:
# Get a Dataframe of just the Category Ids and Category names of the products in the pipeline sheet
pipeline_sheet_category_IDs_and_names = dataframe_to_be_validated.loc[:, "Category ID":"Product Category"]

pipeline_sheet_category_IDs_and_names.head()

,Category ID,Product Category
2,1127,Email Security Software
3,1238,Infrastructure as a Service (IaaS)
4,1037,Marketing Automation Software
5,1780,Virtual Fitting Software
6,1780,Virtual Fitting Software


In [52]:
# Rename the Cat ID column of pipeline sheet to 'URN' to use as a common value for the merge of pipeline and golden dataframes
pipeline_sheet_category_IDs_and_names_renamed = pipeline_sheet_category_IDs_and_names.rename(columns={'Category ID': 'URN'})
pipeline_sheet_category_IDs_and_names_renamed.head()

,URN,Product Category
2,1127,Email Security Software
3,1238,Infrastructure as a Service (IaaS)
4,1037,Marketing Automation Software
5,1780,Virtual Fitting Software
6,1780,Virtual Fitting Software


In [53]:
# Get a Dataframe of just the Category Ids and Category names from the Golden Sheet
golden_sheet_category_IDs_and_names = golden_sheet_dataframe.loc[:, "URN":"Category Name"]
golden_sheet_category_IDs_and_names.head()

,URN,Category Name
0,,
1,1000,Enterprise Accounting Software
2,1001,Campaign Management Software
3,1002,Graphic Design Software
4,1003,Desktop Publishing Software


In [54]:
# Cast merging columns to same dtype so that the merge is effective:
golden_sheet_category_IDs_and_names['URN'] = golden_sheet_category_IDs_and_names['URN'].astype(str)
pipeline_sheet_category_IDs_and_names_renamed['URN'] = pipeline_sheet_category_IDs_and_names_renamed['URN'].astype(str)

# Move index to the dataframe to preserve it after merging dataframes (otherwise would lose index):
pipeline_sheet_category_IDs_and_names_renamed.reset_index(inplace=True)
pipeline_sheet_category_IDs_and_names_renamed.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,index,URN,Product Category
0,2,1127,Email Security Software
1,3,1238,Infrastructure as a Service (IaaS)
2,4,1037,Marketing Automation Software
3,5,1780,Virtual Fitting Software
4,6,1780,Virtual Fitting Software


In [55]:
# Merge both dataframes using inner join on the common URN Column. The index corresponds to the row on the pipeline sheet. NOTE - Not returning doublecoded URNs
golden_and_pipeline_sheets_merged = pd.merge(golden_sheet_category_IDs_and_names, pipeline_sheet_category_IDs_and_names_renamed, how="inner", on=["URN"])

# Get rid of surrounding whitespaces that could alter the comparison:
golden_and_pipeline_sheets_merged['Category Name'] = golden_and_pipeline_sheets_merged['Category Name'].str.strip()
golden_and_pipeline_sheets_merged['Product Category'] = golden_and_pipeline_sheets_merged['Product Category'].str.strip()

golden_and_pipeline_sheets_merged

,URN,Category Name,index,Product Category
0,1000,Enterprise Accounting Software,45,Enterprise Accounting Software
1,1001,Campaign Management Software,131,Campaign Management Software
2,1001,Campaign Management Software,177,Campaign Management Software
3,1001,Campaign Management Software,178,Campaign Management Software
4,1001,Campaign Management Software,381,Campaign Management Software
...,...,...,...,...
436,1806,Cloud Infrastructure Entitlements Management (...,417,Cloud Infrastructure Entitlements Management (...
437,1806,Cloud Infrastructure Entitlements Management (...,418,Cloud Infrastructure Entitlements Management (...
438,1806,Cloud Infrastructure Entitlements Management (...,419,Cloud Infrastructure Entitlements Management (...
439,1806,Cloud Infrastructure Entitlements Management (...,420,Cloud Infrastructure Entitlements Management (...


In [56]:
# Using the merged dataframe locate the rows where the Product Category doesn't match the value of the Category name in the golden sheet 
id_category_mismatch_errs = golden_and_pipeline_sheets_merged.loc[
          ~golden_and_pipeline_sheets_merged.apply(
                          lambda x: x['Category Name'] in x['Product Category'], axis=1)]

id_category_mismatch_errs = id_category_mismatch_errs.set_index('index').sort_index()

id_category_mismatch_errs

,URN,Category Name,Product Category
index,,,
2,1127,Cybersecurity Software,Email Security Software
30,1654,Organizational Design Software,Web Hosting
41,1516,Risk Management Software,Membership Management Software
80,1742,Lead Generation Management Software,Church Management Software
105,1239,Machine Translation Software,Infrastructure as a Service (IaaS)
106,1240,Translation Management Software,Infrastructure as a Service (IaaS)
120,1239,Machine Translation Software,Infrastructure as a Service (IaaS)
146,1127,Cybersecurity Software,Web Hosting
265,1050,Web Content Management (WCM) Systems,Web Content Management Systems


In [57]:
# Get all row numbers that contain a single coded category name - ID error and look them up on the original dataframe:
id_category_mismatch_errs = dataframe_to_be_validated.loc[id_category_mismatch_errs.index]
id_category_mismatch_errs_index = id_category_mismatch_errs.index

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Product Category") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "single_coded_category_error"
highlight_errors(id_category_mismatch_errs_index, column_letter, error)

### Dealing with multi-coded products

##### Preprocessing of the multi-coded columns:

In [ ]:

def preprocess_multi_coded_df(df):
  
  """Function where the preprocessing of the original Catalog + Products pipeline
   sheet dataframe occurs in preparation for error checking. It will be the step previous 
   to going into the logic of the validations of the different types of errors
  (see function below "extract_multi_coded_errors").
  
  Arguments:
    - df:  original Catalog + Products pipeline sheet dataframe
  Returns:
    - original_df_IDs_column: pandas dataframe with only the column "Category ID"; 
    will be used to process the different types of formatting errors
    - multi_coded_categories: dataframe with only products that have been 
    encoded into multiple categories (multiple IDs/category names); will be used
     to check for differing number of elements in both Category IDs columns and
      Category Name, and to get a dataframe clean from formatting errors that 
      will be used further down the line to check if IDs and Category Names match."""

  # Get the column with the category IDs from the original Catalog + Products pipeline sheet:
  pipeline_sheet_prod_cat_id_column = df.loc[:, "Category ID"]

  multi_coded_categories = df.loc[
    # Pick up double and triple coded, cases:
    (pipeline_sheet_prod_cat_id_column.astype(str).str.contains(',') == True)
    ]

  # Encode valid formatted multi-coded columns into lists to check that the num elements matches on both columns:
  multi_coded_categories['Category ID'] = multi_coded_categories['Category ID'].str.split(',', expand=False)
  multi_coded_categories['Product Category'] = multi_coded_categories['Product Category'].str.split('AND', expand=False)

  return [pipeline_sheet_prod_cat_id_column, multi_coded_categories]



def extract_multi_coded_errors(df):
  
  """Main function including the validations of errors for products with 
  multiple encoded categories.
  
  Arguments:
    - df:  original Catalog + Products pipeline sheet dataframe
  Returns:
    - format_and_num_elemen_errs: formatting errors for category IDs and errors 
    for non matching number of elements between Category IDs and Category Names
    """

  # Get column with cat IDs from original df and the preprocessed multi-coded df:
  original_df_IDs_column, multi_coded_df = preprocess_multi_coded_df(df)
  
  # Pick up cases in which num elements of the column Category ID and Product Category are not the same:
  num_elem_each_column = multi_coded_df.applymap(len)
  nonmatching_num_elements = num_elem_each_column.loc[
    ~(num_elem_each_column['Category ID'] ==
      num_elem_each_column['Product Category'])]

  # Get the actual original rows with a mismatch of num elements:
  nonmatching_num_elements_original_rows = multi_coded_df.loc[nonmatching_num_elements.index]

  # [WIP] Find different double coding potential errors in format when inputing 
  # the categories and category IDs:
  format_errs = df.loc[
      # Pick up cases such as '12341348' (no comma): [SEE CELL IMMEDIATELY BELOW]
      (((original_df_IDs_column.astype(str).str.len() >= 8)== True) &
       ((~original_df_IDs_column.astype(str).str.contains(','))== True)) |
      # Pick up cases such as '1234 1348' or '1643 1642,1042' (a missing comma and a whitespace):
      (((original_df_IDs_column.astype(str).str.len() >= 8) == True) &
       ((original_df_IDs_column.astype(str).str.contains(' '))== True)) |
      # Pick up cases such with a whitespace in between IDs such as in '1642, 1042':
      (((original_df_IDs_column.astype(str).str.len() >= 8) == True) &
       ((original_df_IDs_column.astype(str).str.contains(', '))== True)) |
      # Pick up cases in which not a number has been inputted by mistake such as in 'dfs' (it checks if all characters are alphabetic):
      ((original_df_IDs_column.astype(str).str.isalpha())== True)

      # Pick up cases incorrectly formatted by Google Sheets as '1002,1234' into '10,021,234':
      # ---- TODO ----
      ]
  
  # Concat format_errs and errs from non matching num elements in 'Category ID' and 'Product Category':
  format_and_num_elemen_errs = pd.concat([format_errs, nonmatching_num_elements_original_rows])
  # Get rid of duplicate rows for errors:
  format_and_num_elemen_errs = format_and_num_elemen_errs[~format_and_num_elemen_errs.index.duplicated(keep="first")]
  
  return format_and_num_elemen_errs

In [ ]:

def extract_multi_coded_correct_and_errs(df):
  """ 
  Umbrella function that preprocesses, detects formatting errors, and separates 
  them from rows that don't have formatting errors, returning both in different 
  dataframes for further validation processing (checking that ID and category name match).

  Arguments:
    - df:  original Catalog + Products pipeline sheet dataframe.
  Returns: 
    - multi_coded_categories: dataframe with only products that have been 
    encoded into multiple categories (multiple IDs/category names, and that 
    don't contain formatting errors. 
    - format_and_num_elemen_errs: rows that contain such types of errors, 
    with indexes from original dataframe."""
  
  # Get column with cat IDs from original df and the preprocessed multi-coded df:
  _, multi_coded_df = preprocess_multi_coded_df(df)

  format_and_num_elemen_errs = extract_multi_coded_errors(df)

  # Keep only the correctly encoded rows for multiple categories by getting rid of all the errors:
  for err_row in format_and_num_elemen_errs.index:
    try:
      multi_coded_df.drop(err_row, inplace=True)
    except:
      pass


  return [multi_coded_df, format_and_num_elemen_errs]


In [ ]:
# Call the function to get the multi-coded errors
correctly_multi_coded_categories, format_multi_coded_errs = extract_multi_coded_correct_and_errs(
    pipeline_sheet_category_IDs_and_names)

format_multi_coded_errs

In [ ]:
# Move index to the dataframe to preserve it after merging dataframes (otherwise would lose index):
correctly_multi_coded_categories.reset_index(inplace=True)
correctly_multi_coded_categories.head()

In [ ]:
# Separate rows that have several coded IDs/Categories into several rows with 
# sharing indexes (**explode is list dependent**):
correctly_multi_coded_categories = correctly_multi_coded_categories.explode(['Category ID', 'Product Category'])
correctly_multi_coded_categories.head()

In [ ]:
# Rename the Cat ID column to URN to use as a common value for the merge of both dataframes
correctly_multi_coded_categories_renamed = correctly_multi_coded_categories.rename(columns={'Category ID': 'URN'})
correctly_multi_coded_categories_renamed.head()

##### Final merging and checking of multi-coded:

In [ ]:
# Cast merging columns to same dtype so that the merge is effective:
golden_sheet_category_IDs_and_names['URN'] = golden_sheet_category_IDs_and_names['URN'].astype(str)
correctly_multi_coded_categories_renamed['URN'] = correctly_multi_coded_categories_renamed['URN'].astype(str)

In [ ]:
# Merge both dataframes using inner join on the common URN Column. The index corresponds to the row on the pipeline sheet. 
golden_and_pipeline_multi_coded_merged = pd.merge(golden_sheet_category_IDs_and_names, 
                                      correctly_multi_coded_categories_renamed, 
                                      how="inner", on="URN")

golden_and_pipeline_multi_coded_merged.head()

In [ ]:
# Using the multi-coded dataframe to locate the rows where one or more of the Product Category doesn't match the value of the Category name in the golden sheet 
multi_id_category_mismatch_errs = golden_and_pipeline_multi_coded_merged.loc[
                      ~golden_and_pipeline_multi_coded_merged.apply(
                          lambda x: x['Category Name'] in x['Product Category'], axis=1)]

# Set the index to the original values and sort them in ascending order:
multi_id_category_mismatch_errs = multi_id_category_mismatch_errs.set_index('index').sort_index()

multi_id_category_mismatch_errs

##### Gather all errors for Category Names and Category IDs:

In [ ]:
# Get all row numbers that equal an multiple coded format or category mismatch error and look them up on the original dataframe (for consistency):
multi_category_format_errs = dataframe_to_be_validated.loc[format_multi_coded_errs.index]
multi_id_category_mismatch_errs = dataframe_to_be_validated.loc[multi_id_category_mismatch_errs.index]

# Concatenate both types of erros for multi-coded products:
multi_coded_errs = pd.concat([multi_category_format_errs, multi_id_category_mismatch_errs])
multi_coded_errs.head()

In [ ]:
# Concatenate all errors for category names/category IDs (single and multi-coded):
category_IDs_and_names_errs = pd.concat([id_category_mismatch_errs, multi_coded_errs])
category_IDs_and_names_errs_index = category_IDs_and_names_errs.index

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Product Category") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "double_coded_category_error"
highlight_errors(category_IDs_and_names_errs_index, column_letter, error)

## 6. Ensure Product URL is NOT a PDF (i.e. does not end with ‘.pdf’)


In [ ]:
# Get the product urls column
product_urls = dataframe_to_be_validated.loc[:, 'Product URLS']

In [ ]:
# get all product urls that end with .pdf:
product_url_pdf_errs = dataframe_to_be_validated.loc[product_urls.str.endswith('.pdf') == True, :]
product_url_pdf_errs_index = product_url_pdf_errs.index

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Product URLS") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "pdf_error"
highlight_errors(product_url_pdf_errs_index, column_letter, error)

## 7. Ensure product categories assigned to products are NOT Group Representatives

#### Single-coded

In [ ]:
# Get Is group representative column
group_rep_column = golden_sheet_dataframe.loc[:, 'Is group representative']

In [ ]:
# Get all rows where 'is group representative' is set to 'YES'
group_representatives = golden_sheet_dataframe.loc[group_rep_column.str.contains('YES') == True, :]
# Sanity check that we've got the correct amount of group representatives
group_representatives.head()

,,URN,Category Name,Category Description,Aliases,See Also,UPDATED Proposed: Parent Problem Category,parent_product_category_ids,Included in MVP,active,deprecated,is_product_category,is_service_category,is_group_representative,is_solution_category_or_else_problem_category,artifact_id,artifact_name,activity_id,associated_skills_ids,Verified,Published status,adstargeting category,Parent Problem Category Suggested,Is group representative,# MVP Products in Category,# products in category,Example Product,Category Created by,Has Metadata,Evidence,Category in G2?,Only in G2,Notes,,new count,
3,3,1002,Graphic Design Software,Software used to create and edit digital images.,,Vector Graphics Software,Content Management > Content Creation > Graphi...,10009,NO,T,F,T,F,T,T,1000,Software,,,Verified,Published,FALSE,,YES,0,#N/A,,Tea,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,,,Previously Graphics software.,,,
28,31,1031,Customer Support Software,Software used to assist customers with the use...,,,Customer Support,10094,NO,T,F,T,F,T,T,1000,Software,,32157,Verified,Published,TRUE,,YES,0,#N/A,,Tea,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,,,description due to be rewritten (became group ...,,,
45,50,1053,Cloud Security Software,"Software used to protect data, applications, s...",,,Computing > IT System Security > Cloud Security,10040,NO,T,F,T,F,T,T,1000,Software,,,Verified,Published,TRUE,,YES,#VALUE!,#N/A,,Lia,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,,,,,,
112,119,1127,Cybersecurity Software,Software used to protect computer systems and ...,,,Computing > IT System Security,10027,NO,T,F,T,F,T,T,1000,Software,,,Verified,Published,TRUE,,YES,#VALUE!,#N/A,,Lia,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,,,,,,
113,120,1128,Network Security Software,Software used to monitor network settings and ...,,,Computing > IT System Security > Network Security,10039,NO,T,F,T,F,T,T,1000,Software,,,Verified,Published,TRUE,,YES,0,#N/A,,Lia,#N/A,https://docs.google.com/spreadsheets/d/1HIpG4b...,,,,,,


In [ ]:
# Get the urns from the rows of group representatives
group_representative_urns = group_representatives.loc[:, 'URN']
group_rep_frame = group_representative_urns.to_frame().head()

In [ ]:
# Cast merging columns to same dtype so that the merge is effective:
group_representatives['URN'] = group_representatives['URN'].astype(str)
pipeline_sheet_category_IDs_and_names_renamed['URN'] = pipeline_sheet_category_IDs_and_names_renamed['URN'].astype(str)

# Move index to the dataframe to preserve it after merging dataframes (otherwise would lose index):
pipeline_sheet_category_IDs_and_names_renamed.reset_index(inplace=True)
pipeline_sheet_category_IDs_and_names_renamed.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,level_0,index,URN,Product Category
0,0,2,1532,Trucking Software
1,1,3,"1013,1008",Construction Management Software AND Business ...
2,2,4,1280,Network Traffic Analysis (NTA) Tools
3,3,5,1280,Network Traffic Analysis (NTA) Tools
4,4,6,1625,Identity Verification Software


In [ ]:
# Merge both dataframes using inner join on the common URN Column. The index corresponds to the row on the pipeline sheet. NOTE - Not returning doublecoded URNs
group_rep_and_pipeline_sheets_merged = pd.merge(group_representatives, pipeline_sheet_category_IDs_and_names_renamed, how="inner", on=["URN"])
group_rep_err_indexs = group_rep_and_pipeline_sheets_merged['index'].to_numpy()

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Category ID") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "group_rep"
highlight_errors(group_rep_err_indexs, column_letter, error)

#### Multi-Coded

In [ ]:
# Call the function to get the multi-coded errors
correctly_multi_coded_categories, format_multi_coded_errs = extract_multi_coded_correct_and_errs(
    pipeline_sheet_category_IDs_and_names)

correctly_multi_coded_categories

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Category ID,Product Category
3,"[1013, 1008]","[Construction Management Software , Business ..."
11,"[1535, 1290]","[Business Continuity Software , Security Orch..."
12,"[1844, 1200]","[Drug Discovery Software , Data Science & Mac..."
14,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
15,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
17,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
18,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
19,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
20,"[1370, 1844]","[Simulation Software , Drug Discovery Software]"
33,"[1026, 1017]","[E-Commerce Platforms , Customer Relationship..."


In [ ]:
# Separate rows that have several coded IDs/Categories into several rows with 
# sharing indexes (**explode is list dependent**):
correctly_multi_coded_categories_separated = correctly_multi_coded_categories.explode(['Category ID', 'Product Category'])
correctly_multi_coded_categories_separated.head()

,Category ID,Product Category
3,1013,Construction Management Software
3,1008,Business Intelligence (BI) Software
11,1535,Business Continuity Software
11,1290,"Security Orchestration, Automation, and Respo..."
12,1844,Drug Discovery Software


In [ ]:
# Cast merging columns to same dtype so that the merge is effective:
group_representatives['URN'] = group_representatives['URN'].astype(str)
correctly_multi_coded_categories_separated['URN'] = correctly_multi_coded_categories_separated['Category ID'].astype(str)

# Move index to the dataframe to preserve it after merging dataframes (otherwise would lose index):
correctly_multi_coded_categories_separated.reset_index(inplace=True)
correctly_multi_coded_categories_separated.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,index,Category ID,Product Category,URN
0,3,1013,Construction Management Software,1013
1,3,1008,Business Intelligence (BI) Software,1008
2,11,1535,Business Continuity Software,1535
3,11,1290,"Security Orchestration, Automation, and Respo...",1290
4,12,1844,Drug Discovery Software,1844


In [ ]:
# Merge both dataframes using inner join on the common URN Column. The index corresponds to the row on the pipeline sheet. NOTE - Not returning doublecoded URNs
group_rep_and_correctly_multi_coded_merged = pd.merge(group_representatives, correctly_multi_coded_categories_separated, how="inner", on=["URN"])
group_rep_multi_coded_err_indexs = group_rep_and_correctly_multi_coded_merged['index'].to_numpy()
group_rep_multi_coded_err_indexs

array([], dtype=int64)

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Category ID") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "group_rep"
highlight_errors(group_rep_multi_coded_err_indexs, column_letter, error)

## 8. Identify required fields and add a check to ensure all of those fields are filled (except for Product ID which should be left empty) **(Needs to be adjusted for publications sheets/ CHECK WEIRD PRINTING)

<ins>Required fields for new products</ins>: Category ID, Product ID (None), Product Name, isActive, isDeprecated, Product URL, LinkedIn Company URL, Company Name, Source of Product, Locale.

<ins>Required fields for product changes</ins>: Category ID, Product ID, Product Name, isActive, isDeprecated, Product URL, LinkedIn Company URL (for STZ), Company Name, Source of Product, Locale.

The solution will be to create a dataframe for each field with the errors. Then we should be able to index into each of the fields and mark the error independetly.

In [ ]:
# NEW PRODUCTS - Create variables with the list of the names of the columns that 
# will be accessed according to the need to have the field filled-in or left empty:
to_be_filled = ['Category ID', 'Product name', 'Active', 
 'Deprecated', 'Product URLS', 'Comp URL', 'Company Name', 
 'Source of Product', 'Description Locale']
to_leave_empty = 'Product ID'

# Create empty dictionary to store the results:
mandatory_fields = {}

In [ ]:
# Execute this instead of the above only if we're applying the script to product changes or to the publication sheets:
if script_mode_1 == 'product changes' or script_mode_2 == 'publication':

  # PRODUCT CHANGES & PUBLICATION SHEET - (Comp URL filled only for STZ & Product ID filled)
  # Create variables with the list of the names of the columns that will need to be filled:
  to_be_filled = ['Category ID', 'Product name','Product ID', 'Active', 
  'Deprecated', 'Product URLS', 'Comp URL', 'Company Name', 
  'Source of Product', 'Description Locale']

  # Create empty dictionary to store the results:
  mandatory_fields = {}

In [ ]:
import pprint

# initiate indexes object:
mandatory_fields_index = {}

for field in to_be_filled:

  # Get the column data for that column field:
  accessed_column = dataframe_to_be_validated.loc[:, field]

  # For a Company URLs only check if empty when the product is STZ collected (<=30K for the ID):
  if (field == 'Comp URL') and (script_mode_1 == 'product changes' or script_mode_1 == 'publication'):
    mandatory_fields[field] = dataframe_to_be_validated.loc[(accessed_column.eq('') == True)
     & (dataframe_to_be_validated['Product ID'] <= '30000'), :]

    # Get indexes of errors for coloring purposes:
    mandatory_fields_index[field] = mandatory_fields[field ].index

  else:
    # Else, just check in general for blank cells in the provided required fields:
    mandatory_fields[field] = dataframe_to_be_validated.loc[accessed_column.eq('') == True, :]

    # Get indexes of errors for coloring purposes:
    mandatory_fields_index[field] = mandatory_fields[field].index

  # Execute the following after last element in to_be_filled has been processed and only for new products:
  if (field == to_be_filled[-1]) and (script_mode_1 == 'new products' and script_mode_2 == 'pipeline'):  
    # Add the field to be left empty to the dictionary of mandatory fields:
    accessed_column = dataframe_to_be_validated.loc[:, to_leave_empty]
    mandatory_fields[to_leave_empty] = dataframe_to_be_validated.loc[accessed_column.eq('') == False, :]

    # Get indexes of errors for coloring purposes:
    mandatory_fields_index[to_leave_empty] = mandatory_fields[to_leave_empty].index


# Print in a 'pretty', legible way the resulting dict:
pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(mandatory_fields)

mandatory_fields_index

{'Active': Int64Index([], dtype='int64'),
 'Category ID': Int64Index([165], dtype='int64'),
 'Comp URL': Int64Index([], dtype='int64'),
 'Company Name': Int64Index([294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306,
             307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319,
             320, 321, 322, 323, 324, 325],
            dtype='int64'),
 'Deprecated': Int64Index([], dtype='int64'),
 'Description Locale': Int64Index([294, 323, 324], dtype='int64'),
 'Product ID': Int64Index([], dtype='int64'),
 'Product URLS': Int64Index([], dtype='int64'),
 'Product name': Int64Index([], dtype='int64'),
 'Source of Product': Int64Index([], dtype='int64')}

Check the errors for each given field

In [ ]:
if script_mode_1 == 'new products':
  # Execute this only if we're applying the script to new products:
  print("Current script mode:", script_mode_1, "AND", script_mode_2, "\n")
  print(mandatory_fields['Product ID'].index)
else:
  # Execute this for product changes or for publication sheet validations:
  print("Current script mode:", script_mode_1, "AND", script_mode_2, "\n")
  print(mandatory_fields['Product ID'].index)

Current script mode: new products AND pipeline 

Int64Index([], dtype='int64')


In [ ]:
mandatory_fields['Category ID']  

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,
165,Eimear,2022-02-11,,Reputation Management Software,,True Local,,,T,F,,https://www.truelocal.com.au/,en_US,,https://www.linkedin.com/company/truelocal/,"Thryv, Inc",,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,


In [ ]:
mandatory_fields['Product name'] 

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
mandatory_fields['Active'] 

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
mandatory_fields['Deprecated']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
mandatory_fields['Product URLS']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
# This one will only return the blanks for STZ products when we are validating 
# either for product changes or data in the publication sheet (based on logic set earlier):
mandatory_fields['Comp URL']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
mandatory_fields['Company Name']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,
294,Eimear,2022-02-18,1615,Web Hosting,,Nominalia Hosting Wordpress,,,T,F,,https://www.nominalia.com/hosting/wordpress/,,,https://www.linkedin.com/company/nominalia/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
295,Eimear,2022-02-18,1171,Managed Hosting,,Managed WordPress Hosting,,,T,F,,https://www.names.co.uk/wordpress-hosting/mana...,en_UK,,https://www.linkedin.com/company/namesco-limited/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
296,Eimear,2022-02-18,1051,Website Builder Software,,Website Builder,,,T,F,,https://www.names.co.uk/website-builder,en_UK,,https://www.linkedin.com/company/namesco-limited/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
297,Eimear,2022-02-18,1051,Website Builder Software,,Webnode,,,T,F,,https://www.webnode.co.uk/,en_UK,,https://www.linkedin.com/company/webnode-ag/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
298,Eimear,2022-02-18,"1621,1074","Healthcare Analytics Software AND Governance, ...",,Healthverity IPGE Platform,,,T,F,,https://healthverity.com/?utm_campaign=General...,en_US,,https://www.linkedin.com/company/healthverity/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
299,Eimear,2022-02-18,1798,Financial Data APIs,,Yapily Open Banking Data,,,T,F,,https://www.yapily.com/open-banking-data/,en_UK,,https://www.linkedin.com/company/yapily/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
300,Eimear,2022-02-18,1798,Financial Data APIs,,Yapily Open Banking Payment Initiation,,,T,F,,https://www.yapily.com/open-banking-payment-in...,en_UK,,https://www.linkedin.com/company/yapily/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
301,Eimear,2022-02-18,1105,Market Research Software,,Insites Consulting Square,,,T,F,,https://www.insites-consulting.com/platforms/s...,en_US,,https://www.linkedin.com/company/insites-consu...,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
302,Eimear,2022-02-18,"1105,1125",Market Research Software AND Customer Experien...,,Insites Consulting Studio,,,T,F,,https://www.insites-consulting.com/platforms/i...,en_US,,https://www.linkedin.com/company/insites-consu...,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
303,Eimear,2022-02-18,1276,Real Estate CRM Software,,Flueid,,,T,F,,https://www.flueid.com/,en_US,,https://www.linkedin.com/company/flueid/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,


In [ ]:
mandatory_fields['Source of Product']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
mandatory_fields['Description Locale']

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,
294,Eimear,2022-02-18,1615,Web Hosting,,Nominalia Hosting Wordpress,,,T,F,,https://www.nominalia.com/hosting/wordpress/,,,https://www.linkedin.com/company/nominalia/,,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
323,Eimear,2022-02-18,1026,E-Commerce Platforms,,NIDUX,,,T,F,,https://nidux.com/,,,https://www.linkedin.com/company/niduxlatam/,,,,,,,Builtwith Data,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
324,Eimear,2022-02-18,"1455,1865",No-Code Development Platforms AND Student Enga...,,Griddo Digital Campus,,,T,F,,https://griddo.io/digital-campus,,,https://www.linkedin.com/company/griddo/,,,,,,,Builtwith Data,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,


In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
for field in mandatory_fields_index.keys():
    column_index = dataframe_to_be_validated.columns.get_loc(field) 
    column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
    error = "required"
    highlight_errors(mandatory_fields_index[field], column_letter, error)

## 9. Validation that Active = T, and Deprecated = F for New Products, and the opposite deprecations on Product Changes. Also, check that both values are not T/T or F/F.


In [ ]:
# Create a set to collect the errors
active_deprecated_errors_index = set()

In [ ]:
# Get the isActive values of the collected products in the pipeline sheet
is_active_values = dataframe_to_be_validated.loc[:, "Active"]
is_active_values.head()

2    T
3    T
4    T
5    T
6    T
Name: Active, dtype: object

In [ ]:
# Get the rows that don't have T marked in Active column
is_active_errors = dataframe_to_be_validated.loc[is_active_values.str.contains('T') == False, :]
is_active_errors_index = is_active_errors.index

# (Product Changes) Or get the rows that don't have F marked in Active column for to be deprecated products:

if script_mode_1 == 'product changes' and script_mode_2 == 'pipeline':
  field_changed_column = dataframe_to_be_validated.loc[:, "Field Changed"]

  is_active_deprecation_errors = dataframe_to_be_validated.loc[(is_active_values.str.contains('F') == False) & (field_changed_column == 'Product Deprecation'), :]
  is_active_deprecation_errors_index = is_active_deprecation_errors.index

  is_active_errors_index.append(is_active_deprecation_errors_index)

else:
  print(is_active_errors_index)

Int64Index([], dtype='int64')


In [ ]:
# Get the isDeprecated values of the collected products in the pipeline sheet
is_deprecated_values = dataframe_to_be_validated.loc[:, "Deprecated"]
is_deprecated_values.head()

2    F
3    F
4    F
5    F
6    F
Name: Deprecated, dtype: object

In [ ]:
# Get the rows that don't have F marked in Deprecated column
is_deprecated_errors = dataframe_to_be_validated.loc[is_deprecated_values.str.contains('F') == False, :]
is_deprecated_errors_index = is_deprecated_errors.index

# (Product Changes) Or get the rows that don't have T marked in isDeprecated column for to be deprecated products:

if script_mode_1 == 'product changes' and script_mode_2 == 'pipeline':
  is_deprecated_deprecation_errors = dataframe_to_be_validated.loc[(is_deprecated_values.str.contains('T') == False) & (field_changed_column == 'Product Deprecation'), :]
  is_deprecated_deprecation_errors_index = is_deprecated_deprecation_errors.index
  is_deprecated_errors_index.append(is_deprecated_deprecation_errors_index)

else:
  print(is_deprecated_errors_index)

Int64Index([], dtype='int64')


In [ ]:
# (Both New Products and Product Changes) 
# Get rows that have both isActive and isDeprecated is the same value for both (e.g. both 'T' or both 'True'):
conditions = (is_active_values.str.contains('T') & is_deprecated_values.str.contains('T')) | (is_active_values.str.contains('F') & is_deprecated_values.str.contains('F'))
same_value_errors_index = dataframe_to_be_validated.loc[conditions].index
same_value_errors_index

Int64Index([], dtype='int64')

In [ ]:
# Check that the values chosen are consistent for T and F (i.e. not True/TRUE/False/FALSE instead of T and F)
nomenclature_consistency_errors = dataframe_to_be_validated.loc[(is_active_values.str.len() > 1) | (is_deprecated_values.str.len() > 1)]
nomenclature_consistency_errors_index = nomenclature_consistency_errors.index

nomenclature_consistency_errors

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
# Update our error set with our errors from each instance
active_deprecated_errors_index.update(is_active_errors_index, is_deprecated_errors_index, same_value_errors_index, nomenclature_consistency_errors_index)
active_deprecated_errors_index

set()

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Active") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "active_deprecated"
highlight_errors(active_deprecated_errors_index, column_letter, error)

## 10.  Ensure all characters are unicode. (i.e. No special characters like Äô) **[WIP]**


In [ ]:
# Go over all of the columns and make sure there are no "non-standard" characters:

non_ascii_errors_dic = {}  # Initialize dictionary to collect the errors for all the columns

for column in dataframe_to_be_validated.columns:
  print(column)
  current_field = dataframe_to_be_validated[column]
  non_ascii_errors = dataframe_to_be_validated[current_field.str.contains('[^\x00-\x7F]', na=False)]  # na=False in case data in the spreadsheet is not formatted as dtype str
  # non_ascii_errors = dataframe_to_be_validated[current_field.str.contains('([^\x00-\xA8\xAA-\xAD\xAF-\x7F]|)', na=False)] # Accepting trademark and copyright symbols
  # non_ascii_errors = dataframe_to_be_validated[current_field.str.contains('([^\x00-\xA8] | [^\xAA-\xAD] | [^\xAF-\x7F])', na=False)] # Accepting trademark and copyright symbols

  print(non_ascii_errors)

  # add errors to dict:
  non_ascii_errors_dic[column] = non_ascii_errors


# non_ascii_errors_dic

Collected by
Empty DataFrame
Columns: [Collected by, Day added (YYYY-MM-DD), Category ID, Product Category, Product ID, Product name, Product Aliases, Description, Active, Deprecated, Product Skill ID, Product URLS, Description Locale, Company OWNER ID, Comp URL, Company Name, Showcase ID, Showcase URL, Company/product page for display, Add-On: "works for" Product ID, Add-On: "works for" Product Name, Source of Product, Dupe Check- Product Name, Dupe Check- Product URL, Dupe Check- Company ID and Other Company Products, Dupe Check- Staging, Notes/Comments, Moved to GS (Date), Dupe check (Exact Name match), Dupe Check (Exact URL match; Staging), Dupe Check (Exact URL match; Catalog), Reviewer, Reviewed, Error Type, Reviewer Comments, productType]
Index: []
Day added (YYYY-MM-DD)
Empty DataFrame
Columns: [Collected by, Day added (YYYY-MM-DD), Category ID, Product Category, Product ID, Product name, Product Aliases, Description, Active, Deprecated, Product Skill ID, Product URLS, Descript

In [ ]:
# Add coloring and comments to error cells:


## 11.  Ensure Product Skill ID is valid ie. Between 1 - 65000




In [ ]:
# First get the column of product Skill IDs:
product_skills = dataframe_to_be_validated['Product Skill ID'].astype(str)
# Get non-empty values in product skills column
non_empty_product_skills = dataframe_to_be_validated[product_skills.str.len() > 0]
#df = dataframe_to_be_validated[~product_skills.between(1,65000)]
non_empty_product_skills

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,


In [ ]:
# We need to filter out values that contain letters
skills_containing_invalid_chars = non_empty_product_skills[~product_skills.str.isdigit()]
skills_containing_invalid_chars_index = skills_containing_invalid_chars.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
# Convert values to str so that we can use str.contains method:
skills_containing_valid_chars = non_empty_product_skills[product_skills.str.isdigit()]
skill_ids_containing_digits_only = skills_containing_valid_chars['Product Skill ID']
# Catch cases in which Skill ID is not between 1 and 65000:
product_skills_errs = skills_containing_valid_chars.loc[~skill_ids_containing_digits_only.between(1,65000)]
product_skills_errs_index = product_skills_errs.index
#Combine errors from both sections
all_skill_errs_index = product_skills_errs_index.append(skills_containing_invalid_chars_index)
all_skill_errs_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Int64Index([], dtype='int64')

#### Highlight the cells with errors in red

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Product Skill ID") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "skill_id"
highlight_errors(all_skill_errs_index, column_letter, error)

## 12.  Make sure there are no line breaks in product names or descriptions



In [ ]:
dataframe_to_be_validated.head()

,Collected By,Day added (YYYY-MM-DD),Category ID,Product Category,Product ID,Product name,Product Aliases,Product Description,Active,Deprecated,Product Skill ID,Product URLS,Description Locale,Company ID - OWNER,Comp URL,Company Name,Showcase ID,Showcase URL,Company/product page for display,Customer Organizations IDs,Customer Organizations Company URLs,Source of Product,In V1.1,In MVP,Has ingested IMAGES - 2020Dec09,Has ingested VIDEOS - 2020Dec09,Product notes,Dupe Check- Product Name,Dupe Check- Product URL,Dupe Check- Company ID and Other Company Products,Dupe Check- Staging,Notes/Comments,Moved to GS (Date),Dupe check (Exact Name match),Dupe Check (Exact URL match; Staging),Dupe Check (Exact URL match; Catalog),Reviewer,Reviewed,Error type 1,Error type 2,Reviewer Comments,productType,,
2,Eimear,2022-02-04,1532,Trucking Software,,SUPERLOAD,,,T,F,,https://www.bentley.com/en/products/brands/sup...,en_US,,https://www.linkedin.com/company/bentley-systems/,Bentley Systems,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
3,Eimear,2022-02-04,"1013,1008",Construction Management Software AND Business ...,,Bentley SYNCHRO,,,T,F,,https://www.bentley.com/en/products/brands/syn...,en_US,,https://www.linkedin.com/company/bentley-systems/,Bentley Systems,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
4,Eimear,2022-02-04,1280,Network Traffic Analysis (NTA) Tools,,Zeek,,,T,F,,https://corelight.com/about-zeek/how-zeek-work...,en_US,,https://www.linkedin.com/company/corelight/,"Corelight, Inc.",,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
5,Eimear,2022-02-04,1280,Network Traffic Analysis (NTA) Tools,,Corelight Fleet Manager,,,T,F,,https://corelight.com/products/fleet-manager/,en_US,,https://www.linkedin.com/company/corelight/,"Corelight, Inc.",,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,
6,Eimear,2022-02-04,1625,Identity Verification Software,,Prove Pre-Fill,,,T,F,,https://www.prove.com/digital-onboarding,en_US,,https://www.linkedin.com/company/proveidentity/,Prove,,,,,,LSS Top Companies,,,,,,TRUE,TRUE,TRUE,TRUE,,,,,,,FALSE,,,,,,


In [ ]:
all_columns = dataframe_to_be_validated.columns

# Initialize dict for all of the rows with containing linebreaks:
linebreak_errs_index = {}

# Check that there is no line break ("\n") in any of them:
for column in all_columns:

  # Set a condition to make sure Product IDs are processed as str and not int64 dtypes:
  if column == 'Product ID':
    # dataframe_to_be_validated[column] = dataframe_to_be_validated[column].to_string()
    pass
  
  else:
    # **Duplicated column names (e.g. having two columns called 'error type') could throw errors, try to name them differently**
    linebreak_errs_index[column] = dataframe_to_be_validated.loc[dataframe_to_be_validated[column].str.contains('\n', na=False, regex=False)]
    # Get the indexes for the errors (for cell-coloring purposes)s:
    linebreak_errs_index[column] = linebreak_errs_index[column].index

# ** it is returning na for double couded in the category ID field, that is why we are forcing na to be False so that we can still create 
# the mask to access the rows with the errors (otherwise it would throw a cannot mask with nan values error).

In [ ]:
linebreak_errs_index

{'': Int64Index([], dtype='int64'),
 ' ': Int64Index([], dtype='int64'),
 'Active': Int64Index([], dtype='int64'),
 'Category ID': Int64Index([], dtype='int64'),
 'Collected By': Int64Index([], dtype='int64'),
 'Comp URL': Int64Index([], dtype='int64'),
 'Company ID - OWNER': Int64Index([], dtype='int64'),
 'Company Name': Int64Index([], dtype='int64'),
 'Company/product page for display': Int64Index([], dtype='int64'),
 'Customer Organizations Company URLs': Int64Index([], dtype='int64'),
 'Customer Organizations IDs': Int64Index([], dtype='int64'),
 'Day added (YYYY-MM-DD)': Int64Index([], dtype='int64'),
 'Deprecated': Int64Index([], dtype='int64'),
 'Description Locale': Int64Index([], dtype='int64'),
 'Dupe Check (Exact URL match; Catalog)': Int64Index([], dtype='int64'),
 'Dupe Check (Exact URL match; Staging)': Int64Index([], dtype='int64'),
 'Dupe Check- Company ID and Other Company Products': Int64Index([], dtype='int64'),
 'Dupe Check- Product Name': Int64Index([], dtype='int

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
for column in all_columns:
    if column == 'Product ID':  # Until we solve a small bug with that row
      pass
    else:
      column_index = dataframe_to_be_validated.columns.get_loc(column) 
      column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
      error = "linebreak"
      highlight_errors(linebreak_errs_index[column], column_letter, error)

## 13.  Check that 'Showcase URL' and 'Showcase ID' are not placed in each others field

In [ ]:
showcase_url_column, showcase_id_column = dataframe_to_be_validated['Showcase URL'], dataframe_to_be_validated['Showcase ID']

# Get non url data (errors) on the Showcase URL column:
showcase_url_errs = dataframe_to_be_validated[(showcase_url_column.str.startswith('https://www.linkedin.com/showcase') == False) & (showcase_url_column.str.len() > 0)]

# Get url data (errors) on the Showcase ID column:
showcase_id_errs = dataframe_to_be_validated[(showcase_id_column.str.startswith('https://www.linkedin.com/showcase') == True) & (showcase_url_column.str.len() > 0)] 

showcase_url_errs_index = showcase_url_errs.index
showcase_id_errs_index = showcase_id_errs.index
showcase_errors = showcase_url_errs_index.append(showcase_id_errs_index)

In [ ]:
# Highlight all errors in red and highlight the first cell of the row in green for reference
column_index = dataframe_to_be_validated.columns.get_loc("Showcase ID") 
column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
error = "showcase_id"
highlight_errors(showcase_errors, column_letter, error)

## 14.  Check that product name and product ID match for product changes

In [ ]:
# get the product id and names column from the product changes sheet
changes_id_name_columns = dataframe_to_be_validated[['Product ID', 'Product name']]
# Move index to the dataframe to preserve it after merging dataframes (otherwise would lose index):
changes_id_name_columns.reset_index(inplace=True)
changes_id_name_columns.head()

In [ ]:
# get the product id and names column from the product catalog sheet and rename the prod id column to match the changes df so we can merge
catalog_id_name_columns = product_catalog_dataframe[['productId', 'productName']]
catalog_id_name_columns.rename(columns={"productId": "Product ID"}, inplace=True)

In [ ]:
if script_mode_1 == 'product changes':
  # merge dataframes on Product ID column
  product_changes_product_catalog_merged = pd.merge(catalog_id_name_columns, changes_id_name_columns, how="inner", on=["Product ID"])
  product_changes_product_catalog_merged

  # Get rid of surrounding whitespaces that could alter the comparison:
  product_changes_product_catalog_merged['productName'] = product_changes_product_catalog_merged['productName'].str.strip()
  product_changes_product_catalog_merged['Product name'] = product_changes_product_catalog_merged['Product name'].str.strip()

  # Using the merged dataframe locate the rows where the Product names don't match
  product_name_mismatch_errs = product_changes_product_catalog_merged.loc[
            ~product_changes_product_catalog_merged.apply(
                            lambda x: x['productName'] in x['Product name'], axis=1)]

  product_name_mismatch_errs = product_name_mismatch_errs.set_index('index').sort_index()

  product_name_mismatch_errs_index = product_name_mismatch_errs.index
  # Shift the index to match the sheet
  product_name_mismatch_errs.index += 2
  product_name_mismatch_errs_index = product_name_mismatch_errs.index
  product_name_mismatch_errs_index

  # Highlight all errors in red and highlight the first cell of the row in green for reference
  column_index = dataframe_to_be_validated.columns.get_loc("Product name") 
  column_letter = xlsxwriter.utility.xl_col_to_name(column_index)
  error = "product_changes_name_id"
  highlight_errors(product_name_mismatch_errs_index, column_letter, error)

##15. Check that showcase IDs are within the valid range **[WIP]**


## 16.  Dupe Checks **[WIP]**

